In [7]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

tf.reset_default_graph()

input_files = ['../cv/Vd-Orig.png']
dataset = tf.data.Dataset.from_tensor_slices(input_files)
def _parse_function(fn):
    image = tf.read_file(fn)
    image = tf.image.decode_png(image,channels=1)
    image = tf.image.resize_images(image,[100,100])
    image = tf.image.convert_image_dtype(image,tf.float32) # because of the conv layer
    return image

dataset = dataset.map(_parse_function).repeat().batch(1)
iterator = dataset.make_one_shot_iterator()
data = iterator.get_next()

print('Data shape: ',data.shape)


# Workaround for this bug: https://github.com/tensorflow/tensorflow/issues/1325
def get_variable(name):
    for v in tf.trainable_variables():
        #print(v.name, v.shape)
        if v.name == name:
            return v
    raise ValueError('Variable {} not found'.format(name))

original_image = tf.image.encode_png(tf.squeeze(tf.image.convert_image_dtype(data,tf.uint8),[0]))
write_orig = tf.io.write_file('_original.png',original_image)

c = keras.layers.Conv2D(activation='linear',filters=1,kernel_size=(3,3), name='my_conv')(data)
uint8_image = tf.image.encode_png(tf.squeeze(tf.image.convert_image_dtype(c,tf.uint8),[0]))
write_convolved = tf.io.write_file('_convolved.png',uint8_image)

kernel = get_variable('my_conv/kernel:0')
bias = get_variable("my_conv/bias:0")

print('Conv output shape: ',c.shape)
#print(kernel.shape)

kernel_values = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]]).reshape(kernel.shape) * 0.005
init_kernel_op = tf.assign(kernel,kernel_values)

bias_values = np.array([0])
init_bias_op = tf.assign(bias,bias_values)



Data shape:  (?, 100, 100, 1)
Conv output shape:  (?, 98, 98, 1)


In [10]:
with tf.Session() as sess:
    
    sess.run([init_kernel_op,init_bias_op])
    sess.run([write_orig,write_convolved])
    k = sess.run([kernel])
    print(k)

[array([[[[-0.005]],

        [[-0.005]],

        [[-0.005]]],


       [[[-0.005]],

        [[ 0.04 ]],

        [[-0.005]]],


       [[[-0.005]],

        [[-0.005]],

        [[-0.005]]]], dtype=float32)]
